# WORK for P03

## [Work-1]
分類器を評価するスコアにAUCがある。AUCとは何かを調査しパワポ2枚以下でまとめ、次回講義の前日までにslack（generalまたは講師）で提出下さい（パワポまたはPDF）。ファイル名は「P03WK1-yourname」（yournameはあなたの氏名）とすること。<br>
【キーワード】ROC曲線、AUCスコア、分類

--> 多分できる前提...<br>
--> 気になった点あれば解説時にフィードバック予定(個別フィードバックはしない予定)

## [Work-2]
以下ページなどを参考にsklearnにおけるAUCの取得方法を確認せよ。<br>
http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score

--> y_predが1/0（predict）ではなくスコア（predict_proba）であることが混同行列関連評価指標とは違う点を言及予定.

## [Work-3]
以下のコードを下記視点から拡張せよ。拡張したコードは次回講義の前日までにslack（generalか講師）で提出下さい。<br>ファイル名は「P03WK3-yourname」とすること。<br><br>
1) 次の分類アルゴリズムを追加しパイプラインを構築せよ.<br>
　～ランダムフォレスト<br>
　～勾配ブースティング<br>
　～多層パーセプトロン<br>
　～線形SVC（LinearSVCの実装方法は各自調査せよ）<br>
<br>
2) 評価スコアを切り替えてベストモデルが選択できるようにコードを拡張せよ。例えば、AUC基準でベストモデルを決定したい、正解率基準でベストモデルを選択したいというとき、評価基準をGivenとしてパイプラインを性能順に表示させられれば良い。<br>
　～出力形式例：パイプライン名称、評価スコアの2カラム（ただし、評価スコアが指定した評価値になっていること）<br>
　～指定可能な評価指標：正解率、適合率、再現率、F1スコア、AUC<br>
<br>
3)（任意）ベストモデルをpkl形式で所定のフォルダに保存できるようにせよ。

In [7]:
# SET PARAMETER
SCORE = 'recall'

In [9]:
# 以下は拡張前のベースとなるコードです.
# import basice apis
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
m
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# import Data
dataset = load_breast_cancer()
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.DataFrame(dataset.target, columns=['y'])

# Holdout
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

# set Pipelines
pipe_knn = Pipeline([('scl',StandardScaler()), ('est',KNeighborsClassifier())])
pipe_logistic = Pipeline([('scl', StandardScaler()), ('est',LogisticRegression(random_state=1))])
pipe_rf = Pipeline([('scl', StandardScaler()), ('est',RandomForestClassifier(random_state=1))])
pipe_gbc = Pipeline([('scl', StandardScaler()), ('est',GradientBoostingClassifier(random_state=1))])
pipe_mlpc = Pipeline([('scl', StandardScaler()), ('est',MLPClassifier(max_iter=500, random_state=1))])
pipe_svc = Pipeline([('scl', StandardScaler()), ('est',LinearSVC(random_state=1))])




# Evaluation function
def get_model_score(y_true, X_test, pipeline, score_type):
    if score_type == 'auc':
        # "1" stands for the Prob of positive samples for the binary classifcation
        # AUC requries the y_red would be score(probability). 
        y_pred = pipeline.predict_proba(X_test)[:,1]
        score = roc_auc_score(y_true, y_pred)
    elif score_type == 'f1':
        y_pred = pipeline.predict(X_test)
        score = f1_score(y_true, y_pred)
    elif score_type == 'precision':
        y_pred = pipeline.predict(X_test)
        score = precision_score(y_true, y_pred)
    elif score_type == 'accuracy':
        y_pred = pipeline.predict(X_test)
        score = accuracy_score(y_true, y_pred)
    elif score_type == 'recall':
        y_pred = pipeline.predict(X_test)
        score = recall_score(y_true, y_pred)
    else:
        score = None
    return score

# Fit & Evaluation
pipe_names = ['pipe_knn', 'pipe_logistic', 'pipe_rf', 'pipe_gbc', 'pipe_mlpc', 'pipe_svc']
pipe_lines = [pipe_knn, pipe_logistic, pipe_rf, pipe_gbc, pipe_mlpc, pipe_svc]
model_score = []
for (i,pipe) in enumerate(pipe_lines):
    pipe.fit(X_train, y_train.as_matrix().ravel())
    score = get_model_score(y_test.as_matrix().ravel(), X_test, pipe, SCORE)
    print(i, ':', pipe_names[i],'-->',score)
    model_score.append(score)

0 : pipe_knn --> 1.0
1 : pipe_logistic --> 1.0
2 : pipe_rf --> 0.9722222222222222
3 : pipe_gbc --> 1.0
4 : pipe_mlpc --> 1.0
5 : pipe_svc --> 0.9861111111111112


In [11]:
# Get the best model and save
# Reference: http://oppython.hatenablog.com/entry/2016/05/31/205746
index_at_max_score = [i for i, x in enumerate(model_score) if x == max(model_score)]
print(index_at_max_score)
from sklearn.externals import joblib
for index in index_at_max_score:
    pipe_lines[index].fit(X, y.as_matrix().ravel())
    joblib.dump(pipe_lines[index], pipe_names[index] + '.pkl') 

[0, 1, 3, 4]
